In [41]:
import numpy as np
import pandas as pd

from pycaret.classification import *

from joblib import load,dump

# Load Training Processed Data

In [3]:
data = pd.read_csv("../data/train_processed_data.csv")
data.drop("Unnamed: 0",axis=1,inplace=True)
data

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,...,AllServicesActivated,TeleServicesActivated,Connected,gender_Female,gender_Male,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,1.0,1.0,1.0,1.572690,1.0,1.0,2.0,2.0,2.0,2.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0
1,1.0,0.0,0.0,0.717944,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
2,1.0,1.0,1.0,-0.340313,1.0,1.0,1.0,2.0,1.0,2.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0
3,1.0,0.0,1.0,-0.136802,1.0,1.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0
4,1.0,1.0,1.0,-0.381015,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5709,1.0,1.0,0.0,1.572690,1.0,1.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
5710,1.0,0.0,0.0,-0.869441,1.0,1.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
5711,1.0,1.0,0.0,-0.340313,1.0,0.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1
5712,1.0,1.0,0.0,-0.340313,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0


In [10]:
X,y = data.drop("Churn",axis=1),data.Churn

# Setup PyCaret

In [4]:
# Setup PyCaret AutoML
clf = setup(data=data,target="Churn",preprocess=False,train_size=0.9)

,Description,Value
0,Session id,7193
1,Target,Churn
2,Target type,Binary
3,Original data shape,"(5714, 34)"
4,Transformed data shape,"(5714, 34)"
5,Transformed train set shape,"(5142, 34)"
6,Transformed test set shape,"(572, 34)"
7,Numeric features,33


In [33]:
# Get Validation Data from PyCaret Setup
X_valid,y_valid = get_config("X_test"),get_config("y_test")

# Fine Tunning Base Models 

In [5]:
# Load the models
models = ["AdaBoostClassifier","LinearDiscriminantAnalysis","GaussianNB","LogisticRegression","GradientBoostingClassifier"]
for i in range(len(models)):
    models[i] = load(f"../models/{models[i]}.joblib")

In [6]:
# Fine Tunning
best_models = []
for model in models:
    best_models.append(tune_model(model,fold=3))

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8040,0.8521,0.4803,0.6918,0.5670,0.4456,0.4580
1,0.7964,0.8468,0.5502,0.6380,0.5909,0.4563,0.4585
2,0.7900,0.8418,0.4520,0.6551,0.5349,0.4051,0.4167
Mean,0.7968,0.8469,0.4942,0.6616,0.5642,0.4357,0.4444
Std,0.0057,0.0042,0.0413,0.0225,0.0229,0.0221,0.0196


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8046,0.8486,0.5262,0.6713,0.5900,0.4641,0.4701
1,0.8104,0.8468,0.5895,0.6634,0.6243,0.4981,0.4996
2,0.7993,0.8483,0.5240,0.6557,0.5825,0.4526,0.4575
Mean,0.8047,0.8479,0.5466,0.6635,0.5989,0.4716,0.4757
Std,0.0045,0.0008,0.0304,0.0064,0.0182,0.0193,0.0177


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8092,0.8495,0.5218,0.6888,0.5938,0.4722,0.4800
1,0.7952,0.8435,0.5590,0.6321,0.5933,0.4571,0.4587
2,0.8040,0.8466,0.5328,0.6667,0.5922,0.4653,0.4704
Mean,0.8028,0.8465,0.5378,0.6625,0.5931,0.4649,0.4697
Std,0.0058,0.0024,0.0156,0.0233,0.0006,0.0062,0.0087


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


In [45]:
# Store Fine Tuned Models
for model in best_models:
    dump(model,f"../models/finetuned/best_{model.__class__.__name__}")

# Ensemble Methods

## Voting

In [9]:
from sklearn.ensemble import VotingClassifier

In [23]:
voting_clf = VotingClassifier(estimators=list(zip(["AdaBoost","LDA","GaussNB","Logistic","GBoost"],best_models)),voting="soft")
voting_clf.fit(X,y)

VotingClassifier(estimators=[('AdaBoost',
                              AdaBoostClassifier(algorithm='SAMME',
                                                 estimator=None,
                                                 learning_rate=0.4,
                                                 n_estimators=210,
                                                 random_state=3426)),
                             ('LDA',
                              LinearDiscriminantAnalysis(covariance_estimator=None,
                                                         n_components=None,
                                                         priors=None,
                                                         shrinkage=0.0005,
                                                         solver='eigen',
                                                         store_covariance=False,
                                                         tol=0.0001)),
                             ('GaussNB',
                              GaussianNB(priors=None, var_smooth...
                                                         max_depth=3,
                                                         max_features='log2',
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.4,
                                                         min_samples_leaf=5,
                                                         min_samples_split=5,
                                                         min_weight_fraction_leaf=0.0,
                                                         n_estimators=110,
                                                         n_iter_no_change=None,
                                                         random_state=3426,
                                                         subsample=0.4,
                                                         tol=0.0001,
                                                         validation_fraction=0.1,
                                                         verbose=0,
                                                         warm_start=False))],
                 flatten_transform=True, n_jobs=None, verbose=False,
                 voting='soft', weights=None)

In [36]:
# Performace of the Voting Classifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

y_pred = voting_clf.predict(X_valid)
print("Accuracy Score: ",accuracy_score(y_pred,y_valid))
print("Precision Score: ",precision_score(y_pred,y_valid))
print("Recall Score: ",recall_score(y_pred,y_valid))
print("F1 Score: ",f1_score(y_pred,y_valid))

Accuracy Score:  0.798951048951049
Precision Score:  0.6535947712418301
Recall Score:  0.6172839506172839
F1 Score:  0.6349206349206349


In [43]:
# Store it
dump(voting_clf,"../models/finetuned/voting_clf.joblib")

['../models/finetuned/voting_clf.joblib']

## Stacking

In [37]:
from sklearn.ensemble import StackingClassifier

In [38]:
stacking_clf = StackingClassifier(estimators=list(zip(["AdaBoost","LDA","GaussNB","Logistic"],best_models[:-1])),final_estimator=best_models[-1])
stacking_clf.fit(X,y)

StackingClassifier(cv=None,
                   estimators=[('AdaBoost',
                                AdaBoostClassifier(algorithm='SAMME',
                                                   estimator=None,
                                                   learning_rate=0.4,
                                                   n_estimators=210,
                                                   random_state=3426)),
                               ('LDA',
                                LinearDiscriminantAnalysis(covariance_estimator=None,
                                                           n_components=None,
                                                           priors=None,
                                                           shrinkage=0.0005,
                                                           solver='eigen',
                                                           store_covariance=False,
                                                           tol=0.0001)),
                               ('GaussNB',
                                GaussianNB(priors=None,...
                                                              loss='log_loss',
                                                              max_depth=3,
                                                              max_features='log2',
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.4,
                                                              min_samples_leaf=5,
                                                              min_samples_split=5,
                                                              min_weight_fraction_leaf=0.0,
                                                              n_estimators=110,
                                                              n_iter_no_change=None,
                                                              random_state=3426,
                                                              subsample=0.4,
                                                              tol=0.0001,
                                                              validation_fraction=0.1,
                                                              verbose=0,
                                                              warm_start=False),
                   n_jobs=None, passthrough=False, stack_method='auto',
                   verbose=0)

In [40]:
# Performace of the Stacking Classifier
y_pred = stacking_clf.predict(X_valid)
print("Accuracy Score: ",accuracy_score(y_pred,y_valid))
print("Precision Score: ",precision_score(y_pred,y_valid))
print("Recall Score: ",recall_score(y_pred,y_valid))
print("F1 Score: ",f1_score(y_pred,y_valid))

Accuracy Score:  0.8146853146853147
Precision Score:  0.5555555555555556
Recall Score:  0.6910569105691057
F1 Score:  0.6159420289855072


In [44]:
# Store it
dump(stacking_clf,"../models/finetuned/stacking_clf.joblib")

['../models/finetuned/stacking_clf.joblib']

# Best Model

**Voting Classifier** has a higher F1 score and faster,so its the chosen one.

In [46]:
# Check its performance on the Test Set
test_data = pd.read_csv("../data/test_data.csv")
test_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0979-PHULV,Male,0,Yes,Yes,69,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Credit card (automatic),99.45,7007.6,Yes
1,8390-FESFV,Female,0,No,No,62,Yes,Yes,DSL,No,...,Yes,Yes,Yes,Yes,Two year,No,Bank transfer (automatic),84.50,5193.2,No
2,1346-UFHAX,Female,0,No,No,13,Yes,Yes,Fiber optic,Yes,...,No,No,No,No,Month-to-month,Yes,Credit card (automatic),80.00,1029.35,No
3,1741-WTPON,Male,0,Yes,Yes,56,Yes,Yes,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,No,Mailed check,115.85,6567.9,No
4,0410-IPFTY,Female,0,Yes,No,31,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic),83.85,2674.15,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,5419-CONWX,Female,1,No,No,4,Yes,No,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Credit card (automatic),99.80,442.85,Yes
701,4709-LKHYG,Female,0,Yes,Yes,29,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,One year,No,Electronic check,20.00,540.05,No
702,1833-TCXKK,Male,0,Yes,No,45,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),107.75,4882.8,No
703,3923-CSIHK,Female,1,Yes,No,50,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,71.05,3444.85,Yes


Process it using the **Pipeline**<br>
We will use python file for this cause pipeline is saved in python file and notebook file cant interacte with it.<br>

Check **scripts/process_test_data.py**

In [105]:
test_processed_data = pd.read_csv("../data/test_processed_data.csv")
# Add Churn to the processed test data from the original test data
from sklearn.preprocessing import LabelEncoder
test_processed_data["Churn"] = LabelEncoder().fit_transform(pd.read_csv("../data/test_data.csv")["Churn"])

In [106]:
X_test,y_test = test_processed_data.drop("Churn",axis=1),test_processed_data["Churn"]

In [107]:
# Performace On the Test Data
y_pred = voting_clf.predict(X_test)
print("Accuracy Score: ",accuracy_score(y_pred,y_test))
print("Precision Score: ",precision_score(y_pred,y_test))
print("Recall Score: ",recall_score(y_pred,y_test))
print("F1 Score: ",f1_score(y_pred,y_test))

Accuracy Score:  0.8014184397163121
Precision Score:  0.6577540106951871
Recall Score:  0.6180904522613065
F1 Score:  0.6373056994818653


**Acceptable**